In [2]:
import pandas as pd
import  random
import schedule
import time
import numpy as np
from math import floor
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
from datetime import datetime, date, timedelta
from faker import Faker

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP" #"HIST_ATUALIZACOES"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring, pool_size=10, max_overflow=30)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print(f"Query teste executada com sucesso, conexão com o banco de dados {bancodados} bem-sucedida")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
testar_conexao_sqlalchemy(engine) 

def job_add_notas():
    qtd_packs = 1
    
    for i in range(qtd_packs): #1000 notas por pack
        num_notas = random.randint(188,512)

        lista_notas_fiscais = []
        lista_ids_nfs = []

        df_tabela_produto = pd.read_sql("SELECT IDPRODUTO,PRECO_CUSTO,PRECO_UNIT FROM PRODUTO",engine)
        id_produto = df_tabela_produto['IDPRODUTO'].count()
        lista_clientes = (pd.read_sql("SELECT IDCLIENTE FROM CLIENTE", engine)['IDCLIENTE']).to_list()
        lista_lojas = (pd.read_sql("SELECT IDLOJA FROM LOJA", engine)['IDLOJA']).to_list()
        lista_forma_pagamento = (pd.read_sql("SELECT IDFORMAPGMT FROM FORMA_PAGAMENTO", engine)['IDFORMAPGMT']).to_list()
        lista_id_cfop = (pd.read_sql("SELECT IDCODCFOP FROM CFOP", engine)['IDCODCFOP']).to_list()
        lista_chave_acesso_db = (pd.read_sql("SELECT CHAVE_ACESSO FROM NOTAFISCAL", engine)['CHAVE_ACESSO']).to_list()

        #display(df_tabela_produto)

        qtde_nf_no_bancodados = (engine.connect().execute(text("SELECT COUNT(IDNOTAFISCAL) FROM NOTAFISCAL")).scalar()) + 1000

        #DATA DA NOTA
            #data da nota
            #numero_dias = floor((id_notafiscal/100) + 1)
            #data_minima = datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))
            #data_maxima = (datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))) + timedelta(days=1)
            #data_nota_ini = fake.date_time_between(start_date=(datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))), end_date=((datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))) + timedelta(days=1)))
            #
            #hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
            #minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
            #segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
            #data_nota = (data_nota_ini).replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio)


        #data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
        #hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
        #minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
        #segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
        #data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado


        for _ in range(num_notas):

            id_notafiscal = qtde_nf_no_bancodados + 1
            qtde_itens_nf = int(np.random.choice([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
                                                  21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40],
                                                  size=None,replace=True,
                                                  p=[0.12 ,0.099,0.07,0.065,0.07,0.014,0.012,0.081,0.011,0.099,
                                                     0.081,0.012,0.012,0.011,0.012,0.018,0.009,0.009,0.008,0.008,
                                                     0.017,0.006,0.006,0.017,0.015,0.014,0.004,0.003,0.007,0.009,
                                                     0.019,0.002,0.007,0.011,0.013,0.004,0.004,0.003,0.008,0.01]))

            numero_dias = floor((id_notafiscal/200) + 1)
            data_minima = datetime(2020,1,1) + (timedelta(days=(numero_dias)))
            data_maxima = (datetime(2020,1,1) + (timedelta(days=(numero_dias)))) + timedelta(days=1)
            data_nota_ini = fake.date_time_between(start_date=data_minima, end_date=data_maxima)
            hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
            minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
            segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
            data_nota = (data_nota_ini).replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio)
            
            #data_nota = (fake.date_time_between(start_date=(datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/(random.randint(193, 857))) + 1))))), end_date=((datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))) + timedelta(days=1)))).replace(hour=(random.randint(7, 21)), minute=(random.randint(0, 59)), second=(random.randint(0, 59)))

            for _ in range(qtde_itens_nf):
                id_produto = random.randint(1,(df_tabela_produto['IDPRODUTO'].count()))
                quantidade_produtos = random.randint(1,10)
                preco_unit = round(df_tabela_produto.loc[df_tabela_produto["IDPRODUTO"] == id_produto, "PRECO_UNIT"].values[0],2)
                total_item = round((quantidade_produtos*preco_unit), 2)
                data_criacao = datetime.today().date()
                data_atlz = None
                versao = 1        

                lista_notas_fiscais.append((id_notafiscal, data_nota, id_produto, quantidade_produtos, preco_unit, total_item, data_criacao, data_atlz, versao))

            id_cliente = random.choice(lista_clientes)
            id_loja = random.choice(lista_lojas)
            id_forma_pgmt = np.random.choice(lista_forma_pagamento,size=None, replace=True, p=[0.15,0.23,0.22,0.165,0.13,0.08,0.005,0.02])    #random.choice(lista_forma_pagamento)
            id_cfop = random.choice(lista_id_cfop)
            while True:
                chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
                if chaveacesso not in lista_chave_acesso_db:
                    break
            lista_ids_nfs.append((id_notafiscal, id_cliente, id_loja, id_forma_pgmt, id_cfop, chaveacesso))
            qtde_nf_no_bancodados = qtde_nf_no_bancodados + 1

        df_itemnota = pd.DataFrame(lista_notas_fiscais, columns=["ID_NOTAFISCAL", "DATA","ID_PRODUTO","QUANTIDADE","PRECO_UNIT",
                                                                    "TOTAL_ITEM", "DATA_CRIACAO","DATA_ATLZ","VERSAO"])
        df_ids_nfs = pd.DataFrame(lista_ids_nfs, columns=["ID_NOTAFISCAL", "ID_CLIENTE","ID_LOJA","ID_FORMA_PGMT","ID_CFOP",
                                                          "CHAVE_ACESSO"])

        df_agrupado = df_itemnota.groupby("ID_NOTAFISCAL").agg({
                                                                    "DATA": "first",  

                                                                    "DATA_CRIACAO": "first",  
                                                                    "DATA_ATLZ": "first",  
                                                                    "VERSAO": "first"
        }).reset_index()
        df_total_nota = round((df_itemnota.groupby("ID_NOTAFISCAL")["TOTAL_ITEM"].sum().reset_index()), 2)
        df_total_nota.columns = ["ID_NOTAFISCAL", "TOTAL_NOTA"]
        df_datas_notas = df_itemnota.groupby("ID_NOTAFISCAL")["DATA"].first().reset_index()
        df_datas_notas.columns = ["ID_NOTAFISCAL", "DATA"]

        df_nota_fiscal = pd.merge(df_total_nota, df_agrupado, on="ID_NOTAFISCAL", how="left")
        df_nota_fiscal = pd.merge(df_nota_fiscal, df_ids_nfs, on="ID_NOTAFISCAL", how="left")

        df_itemnota = pd.merge(df_itemnota, df_ids_nfs, on="ID_NOTAFISCAL", how="left")
        df_itemnota = pd.merge(df_itemnota, df_total_nota, on="ID_NOTAFISCAL", how="left")

        df_ITEMNOTAf = pd.DataFrame({
                        'ID_NOTAFISCAL':  df_itemnota['ID_NOTAFISCAL'],
                        'ID_PRODUTO':     df_itemnota['ID_PRODUTO'],
                        'QUANTIDADE':     df_itemnota['QUANTIDADE'],
                        'PRECO_UNIT':     df_itemnota['PRECO_UNIT'],
                        'TOTAL_ITEM':     df_itemnota['TOTAL_ITEM'],
                        'DATA_CRIACAO':   df_itemnota['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_itemnota['DATA_ATLZ'],
                        'VERSAO':         df_itemnota['VERSAO'],
        })

        df_NOTAFISCALf = pd.DataFrame({
                        'DATA':           df_nota_fiscal['DATA'],
                        'NUMERONF':       df_nota_fiscal['ID_NOTAFISCAL'],
                        'TOTAL_NOTA':     df_nota_fiscal['TOTAL_NOTA'],
                        'ID_FORMA_PGMT':  df_nota_fiscal['ID_FORMA_PGMT'],
                        'ID_CLIENTE':     df_nota_fiscal['ID_CLIENTE'],
                        'ID_LOJA':        df_nota_fiscal['ID_LOJA'],
                        'ID_CFOP':        df_nota_fiscal['ID_CFOP'],
                        'CHAVE_ACESSO':   df_ids_nfs['CHAVE_ACESSO'],
                        'DATA_CRIACAO':   df_nota_fiscal['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_nota_fiscal['DATA_ATLZ'],
                        'VERSAO':         df_nota_fiscal['VERSAO'],
        })

        df_VENDAf = pd.DataFrame({
                        'DATA':           df_itemnota['DATA'],
                        'ID_NOTAFISCAL':  df_itemnota['ID_NOTAFISCAL'],
                        'ID_CLIENTE':     df_itemnota['ID_CLIENTE'],
                        'ID_LOJA':        df_itemnota['ID_LOJA'],
                        'ID_FORMA_PGMT':  df_itemnota['ID_FORMA_PGMT'],
                        'TOTAL_VENDA':    df_itemnota['TOTAL_NOTA'],
                        'DATA_CRIACAO':   df_itemnota['DATA_CRIACAO'],
                        'DATA_ATLZ':      df_itemnota['DATA_ATLZ'],
                        'VERSAO':         df_itemnota['VERSAO'],
        })

        #Envio das NFs, ITEMNOTA e VENDA para o banco de dados

        tipo_dados_ITEMNOTAf = {
                        'ID_NOTAFISCAL':  INTEGER,
                        'ID_PRODUTO':     INTEGER,
                        'QUANTIDADE':     INTEGER,
                        'PRECO_UNIT':     NUMERIC(18,2),
                        'TOTAL_ITEM':     NUMERIC(18,2),
                        'DATA_CRIACAO':   DATE,
                        'DATA_ATLZ':      DATE,
                        'VERSAO':         INTEGER,
        }

        tipo_dados_NOTAFISCALf = {
                        'DATA':           DATE,
                        'NUMERONF':       INTEGER,
                        'TOTAL_NOTA':     NUMERIC(18,2),
                        'ID_FORMA_PGMT':  INTEGER,
                        'ID_CLIENTE':     INTEGER,
                        'ID_LOJA':        INTEGER,
                        'ID_CFOP':        INTEGER,
                        'CHAVE_ACESSO':   VARCHAR(100),
                        'DATA_CRIACAO':   DATE,
                        'DATA_ATLZ':      DATE,
                        'VERSAO':         INTEGER,
        }

        tipo_dados_VENDAf = {
                        'DATA':           DATE,
                        'ID_NOTAFISCAL':  INTEGER,
                        'ID_CLIENTE':     INTEGER,
                        'ID_LOJA':        INTEGER,
                        'ID_FORMA_PGMT':  INTEGER,
                        'TOTAL_VENDA':    NUMERIC(18,2),
                        'DATA_CRIACAO':   DATE,
                        'DATA_ATLZ':      DATE,
                        'VERSAO':         INTEGER,
        }

        try:
            df_NOTAFISCALf.to_sql(name='NOTAFISCAL', con=engine,if_exists='append',index=False,dtype=tipo_dados_NOTAFISCALf)
            df_ITEMNOTAf.to_sql(  name='ITEMNOTA',   con=engine,if_exists='append',index=False,dtype=tipo_dados_ITEMNOTAf)
            df_VENDAf.to_sql(     name='VENDA',      con=engine,if_exists='append',index=False,dtype=tipo_dados_VENDAf)
            #print(f"Notas adicionadas ao banco de dados com sucesso.")
        except Exception as e:
            print(f"Erro ao conectar ao banco de dados: {e}")

    print(f"Notas adicionadas ao banco de dados com sucesso.")

#schedule.every(10).seconds.do(job_add_notas)
#print("Script iniciado. Pressione Ctrl+C para parar.")

try:
    while True:
        job_add_notas()
        time.sleep(1)  
except KeyboardInterrupt:
    print("\nScript interrompido pelo usuário.")

job_add_notas()


Query teste executada com sucesso, conexão com o banco de dados MERCADO_OLTP bem-sucedida
Notas adicionadas ao banco de dados com sucesso.
Erro ao conectar ao banco de dados: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)
Notas adicionadas ao banco de dados com sucesso.
Notas adicionadas ao banco de dados com sucesso.
Notas adicionadas ao banco de dados com sucesso.

Script interrompido pelo usuário.
Notas adicionadas ao banco de dados com sucesso.


In [59]:
import pandas as pd
import  random
from math import floor
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
from datetime import datetime, date, timedelta
from faker import Faker

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP" #"HIST_ATUALIZACOES"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring, pool_size=10, max_overflow=30)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print(f"Query teste executada com sucesso, conexão com o banco de dados {bancodados} bem-sucedida")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
testar_conexao_sqlalchemy(engine) 

qtd_packs = 10

for i in range(qtd_packs):
    num_notas = 100

    lista_notas_fiscais = []
    lista_ids_nfs = []

    df_tabela_produto = pd.read_sql("SELECT IDPRODUTO,PRECO_CUSTO,PRECO_UNIT FROM PRODUTO",engine)
    id_produto = df_tabela_produto['IDPRODUTO'].count()
    lista_clientes = (pd.read_sql("SELECT IDCLIENTE FROM CLIENTE", engine)['IDCLIENTE']).to_list()
    lista_lojas = (pd.read_sql("SELECT IDLOJA FROM LOJA", engine)['IDLOJA']).to_list()
    lista_forma_pagamento = (pd.read_sql("SELECT IDFORMAPGMT FROM FORMA_PAGAMENTO", engine)['IDFORMAPGMT']).to_list()
    lista_id_cfop = (pd.read_sql("SELECT IDCODCFOP FROM CFOP", engine)['IDCODCFOP']).to_list()
    lista_chave_acesso_db = (pd.read_sql("SELECT CHAVE_ACESSO FROM NOTAFISCAL", engine)['CHAVE_ACESSO']).to_list()

    #display(df_tabela_produto)

    qtde_nf_no_bancodados = (conn_TESTE_NF.connect().execute(text("SELECT COUNT(IDNOTAFISCAL) FROM NOTAFISCAL")).scalar()) + 1000

    #DATA DA NOTA
        #data da nota
        #numero_dias = floor((id_notafiscal/100) + 1)
        #data_minima = datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))
        #data_maxima = (datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))) + timedelta(days=1)
        #data_nota_ini = fake.date_time_between(start_date=(datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))), end_date=((datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))) + timedelta(days=1)))
        #
        #hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
        #minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
        #segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
        #data_nota = (data_nota_ini).replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio)


    data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
    hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
    minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
    segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
    data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado


    for _ in range(num_notas):

        id_notafiscal = qtde_nf_no_bancodados + 1
        qtde_itens_nf = int(random.randint(1,10))
        data_nota = (fake.date_time_between(start_date=(datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))), end_date=((datetime(2020,1,1) + (timedelta(days=(floor((id_notafiscal/100) + 1))))) + timedelta(days=1)))).replace(hour=(random.randint(7, 21)), minute=(random.randint(0, 59)), second=(random.randint(0, 59)))

        for _ in range(qtde_itens_nf):
            id_produto = random.randint(1,(df_tabela_produto['IDPRODUTO'].count()))
            quantidade_produtos = random.randint(1,10)
            preco_unit = round(df_tabela_produto.loc[df_tabela_produto["IDPRODUTO"] == id_produto, "PRECO_UNIT"].values[0],2)
            total_item = round((quantidade_produtos*preco_unit), 2)
            data_criacao = datetime.today().date()
            data_atlz = None
            versao = 1        

            lista_notas_fiscais.append((id_notafiscal, data_nota, id_produto, quantidade_produtos, preco_unit, total_item, data_criacao, data_atlz, versao))

        id_cliente = random.choice(lista_clientes)
        id_loja = random.choice(lista_lojas)
        id_forma_pgmt = random.choice(lista_forma_pagamento)
        id_cfop = random.choice(lista_id_cfop)
        while True:
            chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
            if chaveacesso not in lista_chave_acesso_db:
                break
        lista_ids_nfs.append((id_notafiscal, id_cliente, id_loja, id_forma_pgmt, id_cfop, chaveacesso))
        qtde_nf_no_bancodados = qtde_nf_no_bancodados + 1

    df_itemnota = pd.DataFrame(lista_notas_fiscais, columns=["ID_NOTAFISCAL", "DATA","ID_PRODUTO","QUANTIDADE","PRECO_UNIT",
                                                                "TOTAL_ITEM", "DATA_CRIACAO","DATA_ATLZ","VERSAO"])
    df_ids_nfs = pd.DataFrame(lista_ids_nfs, columns=["ID_NOTAFISCAL", "ID_CLIENTE","ID_LOJA","ID_FORMA_PGMT","ID_CFOP",
                                                      "CHAVE_ACESSO"])

    df_agrupado = df_itemnota.groupby("ID_NOTAFISCAL").agg({
                                                                "DATA": "first",  

                                                                "DATA_CRIACAO": "first",  
                                                                "DATA_ATLZ": "first",  
                                                                "VERSAO": "first"
    }).reset_index()
    df_total_nota = round((df_itemnota.groupby("ID_NOTAFISCAL")["TOTAL_ITEM"].sum().reset_index()), 2)
    df_total_nota.columns = ["ID_NOTAFISCAL", "TOTAL_NOTA"]
    df_datas_notas = df_itemnota.groupby("ID_NOTAFISCAL")["DATA"].first().reset_index()
    df_datas_notas.columns = ["ID_NOTAFISCAL", "DATA"]

    df_nota_fiscal = pd.merge(df_total_nota, df_agrupado, on="ID_NOTAFISCAL", how="left")
    df_nota_fiscal = pd.merge(df_nota_fiscal, df_ids_nfs, on="ID_NOTAFISCAL", how="left")

    df_itemnota = pd.merge(df_itemnota, df_ids_nfs, on="ID_NOTAFISCAL", how="left")
    df_itemnota = pd.merge(df_itemnota, df_total_nota, on="ID_NOTAFISCAL", how="left")

    df_ITEMNOTAf = pd.DataFrame({
                    'ID_NOTAFISCAL':  df_itemnota['ID_NOTAFISCAL'],
                    'ID_PRODUTO':     df_itemnota['ID_PRODUTO'],
                    'QUANTIDADE':     df_itemnota['QUANTIDADE'],
                    'PRECO_UNIT':     df_itemnota['PRECO_UNIT'],
                    'TOTAL_ITEM':     df_itemnota['TOTAL_ITEM'],
                    'DATA_CRIACAO':   df_itemnota['DATA_CRIACAO'],
                    'DATA_ATLZ':      df_itemnota['DATA_ATLZ'],
                    'VERSAO':         df_itemnota['VERSAO'],
    })

    df_NOTAFISCALf = pd.DataFrame({
                    'DATA':           df_nota_fiscal['DATA'],
                    'NUMERONF':       df_nota_fiscal['ID_NOTAFISCAL'],
                    'TOTAL_NOTA':     df_nota_fiscal['TOTAL_NOTA'],
                    'ID_FORMA_PGMT':  df_nota_fiscal['ID_FORMA_PGMT'],
                    'ID_CLIENTE':     df_nota_fiscal['ID_CLIENTE'],
                    'ID_LOJA':        df_nota_fiscal['ID_LOJA'],
                    'ID_CFOP':        df_nota_fiscal['ID_CFOP'],
                    'CHAVE_ACESSO':   df_ids_nfs['CHAVE_ACESSO'],
                    'DATA_CRIACAO':   df_nota_fiscal['DATA_CRIACAO'],
                    'DATA_ATLZ':      df_nota_fiscal['DATA_ATLZ'],
                    'VERSAO':         df_nota_fiscal['VERSAO'],
    })

    df_VENDAf = pd.DataFrame({
                    'DATA':           df_itemnota['DATA'],
                    'ID_NOTAFISCAL':  df_itemnota['ID_NOTAFISCAL'],
                    'ID_CLIENTE':     df_itemnota['ID_CLIENTE'],
                    'ID_LOJA':        df_itemnota['ID_LOJA'],
                    'ID_FORMA_PGMT':  df_itemnota['ID_FORMA_PGMT'],
                    'TOTAL_VENDA':    df_itemnota['TOTAL_NOTA'],
                    'DATA_CRIACAO':   df_itemnota['DATA_CRIACAO'],
                    'DATA_ATLZ':      df_itemnota['DATA_ATLZ'],
                    'VERSAO':         df_itemnota['VERSAO'],
    })

    #Envio das NFs, ITEMNOTA e VENDA para o banco de dados
    bancodados2 = "TESTE_NF" 
    conexaostring_TESTE_NF = f"mssql+pyodbc://{server}/{bancodados2}?driver={driver}&Trusted_Connection=yes"
    conn_TESTE_NF = create_engine(conexaostring_TESTE_NF, pool_size=10, max_overflow=30)

    #Função para testar a conexão
    def testar_conexao_sqlalchemy(conn_TESTE_NF):
        try:        
            with conn_TESTE_NF.connect() as conn: #Tenta conectar ao banco de dados
                result = conn.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
                #print(f"Query teste executada com sucesso, conexão com o banco de dados {bancodados2} bem-sucedida")
                return True
        except Exception as e:
            print(f"Erro ao conectar ao banco de dados: {e}")
            return False
    #testa a conexão
    testar_conexao_sqlalchemy(conn_TESTE_NF) 

    df_teste = pd.read_sql("SELECT * FROM NOTAFISCAL",conn_TESTE_NF)

    tipo_dados_ITEMNOTAf = {
                    'ID_NOTAFISCAL':  INTEGER,
                    'ID_PRODUTO':     INTEGER,
                    'QUANTIDADE':     INTEGER,
                    'PRECO_UNIT':     NUMERIC(18,2),
                    'TOTAL_ITEM':     NUMERIC(18,2),
                    'DATA_CRIACAO':   DATE,
                    'DATA_ATLZ':      DATE,
                    'VERSAO':         INTEGER,
    }

    tipo_dados_NOTAFISCALf = {
                    'DATA':           DATE,
                    'NUMERONF':       INTEGER,
                    'TOTAL_NOTA':     NUMERIC(18,2),
                    'ID_FORMA_PGMT':  INTEGER,
                    'ID_CLIENTE':     INTEGER,
                    'ID_LOJA':        INTEGER,
                    'ID_CFOP':        INTEGER,
                    'CHAVE_ACESSO':   VARCHAR(100),
                    'DATA_CRIACAO':   DATE,
                    'DATA_ATLZ':      DATE,
                    'VERSAO':         INTEGER,
    }

    tipo_dados_VENDAf = {
                    'DATA':           DATE,
                    'ID_NOTAFISCAL':  INTEGER,
                    'ID_CLIENTE':     INTEGER,
                    'ID_LOJA':        INTEGER,
                    'ID_FORMA_PGMT':  INTEGER,
                    'TOTAL_VENDA':    NUMERIC(18,2),
                    'DATA_CRIACAO':   DATE,
                    'DATA_ATLZ':      DATE,
                    'VERSAO':         INTEGER,
    }

    try:
        df_NOTAFISCALf.to_sql(name='NOTAFISCAL',con=conn_TESTE_NF,if_exists='append',index=False,dtype=tipo_dados_NOTAFISCALf)
        df_ITEMNOTAf.to_sql(name='ITEMNOTA',con=conn_TESTE_NF,if_exists='append',index=False,dtype=tipo_dados_ITEMNOTAf)
        df_VENDAf.to_sql(name='VENDA',con=conn_TESTE_NF,if_exists='append',index=False,dtype=tipo_dados_VENDAf)
        print(f"Notas adicionadas ao banco de dados com sucesso. Nf inicial:{qtde_nf_no_bancodados} Nf final:{(qtde_nf_no_bancodados+num_notas)}")
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")



Query teste executada com sucesso, conexão com o banco de dados MERCADO_OLTP bem-sucedida
Notas adicionadas ao banco de dados com sucesso. Nf inicial:882540 Nf final:882640
Notas adicionadas ao banco de dados com sucesso. Nf inicial:882640 Nf final:882740
Notas adicionadas ao banco de dados com sucesso. Nf inicial:882740 Nf final:882840
Notas adicionadas ao banco de dados com sucesso. Nf inicial:882840 Nf final:882940
Notas adicionadas ao banco de dados com sucesso. Nf inicial:882940 Nf final:883040
Notas adicionadas ao banco de dados com sucesso. Nf inicial:883040 Nf final:883140
Notas adicionadas ao banco de dados com sucesso. Nf inicial:883140 Nf final:883240
Notas adicionadas ao banco de dados com sucesso. Nf inicial:883240 Nf final:883340
Notas adicionadas ao banco de dados com sucesso. Nf inicial:883340 Nf final:883440
Notas adicionadas ao banco de dados com sucesso. Nf inicial:883440 Nf final:883540
